# Fourier -- check

In [ ]:
%reload_ext cpp2py.magic

In [ ]:
%%cpp2py -C pytriqs
#include <triqs/gfs.hpp>
using namespace triqs::gfs;

// The type of a Green function : (k,omega) -> Complex number
using g_k_w_type = gf_view<cartesian_product<brillouin_zone, imfreq>, scalar_valued>;
using g_r_t_type = gf<cartesian_product<cyclic_lattice, imtime>, scalar_valued>;

g_k_w_type bubble(g_k_w_type g0) {
    
    // Fourier Transformation of k, \omega to obtain g(r,t)
    auto grt = make_gf_from_fourier<0,1>(g0);
    
    // The mesh of gtr is a cartesian product mt x mr. We decompose it.
    auto [mr, mt] = grt.mesh();
    
    // The inverse temperature from the mesh
    double beta = mt.domain().beta;
    
    // A new mesh for chi, with a bosonic statistics, but same size as mt.
    auto mtb = gf_mesh<imtime>{beta, Boson, mt.size()};
    
    // Build chi (r, tau) with this new mesh.
    auto chi0 = g_r_t_type{{mr, mtb}};

    // we fill chi : chi(tau, r) = g(beta - tau, -r) * g(tau, r)
    for (auto const &r : mr)      
        for (auto const &t : mtb) 
            chi0[r, t] = grt(-r, beta - t) * grt(r, t); 

    // Fourier transform back to k, \omega space and return
    return make_gf_from_fourier<0,1>(chi0);
}

In [ ]:
%%cpp2py -C pytriqs
#include <triqs/gfs.hpp>
using namespace triqs::gfs;

// The type of a Green function : (k,omega) -> Complex number
using g_k_w_type = gf_view<cartesian_product<brillouin_zone, imfreq>, scalar_valued>;
using g_r_t_type = gf<cartesian_product<cyclic_lattice, imtime>, scalar_valued>;

g_r_t_type four(g_k_w_type g) {
    
    // Fourier Transformation of k, \omega to obtain g(r,t)
    return make_gf_from_fourier<0,1>(g);
}

In [ ]:
from pytriqs.plot.mpl_interface import plt, oplot
import numpy as np
from pytriqs.lattice import BravaisLattice, BrillouinZone
from pytriqs.gf import MeshBrillouinZone, MeshImFreq, Gf, MeshProduct, Idx
from pytriqs.gf import inverse, MeshImTime, MeshCyclicLattice, Fourier, InverseFourier
from pytriqs.archive import HDFArchive
from scipy.optimize import fsolve

## Parameters

In [ ]:
beta = 10.
mu = 0.0 # Should we adjust to get correct density?

t = 1.0
tp = 0.175 * t
tpp = 0.05 * t

#U = 5.75 * t
U = 2. * t

n_k = 4
n_w = 8*1024
iGamma = 0.01 * 1j

BL = BravaisLattice([(1, 0, 0), (0, 1, 0)])
BZ = BrillouinZone(BL)
kmesh = MeshBrillouinZone(BZ, n_k = n_k)

## Functions

In [ ]:
# TODO: put the right dispersion here !!!
def eps(kx,ky):
    return -2 * t* (np.cos(kx) + np.cos(ky)) - 4*tp*np.cos(kx)*np.cos(ky) # + ......

def get_g0(beta):
    wmesh = MeshImFreq(beta=beta, S='Fermion', n_max=n_w)
    w = np.tensordot(np.ones(n_k*n_k), list(wmesh.values()), 0)
    k = np.tensordot(list(kmesh.values()), np.ones(2*n_w), 0)
    kx, ky = k[:,0,:], k[:,1,:]

    g0 = Gf(mesh = MeshProduct(kmesh, wmesh), target_shape = [])
    g0.data[:,:] = 1 / (w - eps(kx,ky) + iGamma)
    return g0

def get_chi0(g0):    
    return bubble(g0)

def chi_rpa(chi0_wk, U):
    chi_rpa = chi0_kw.copy()
    chi_rpa = chi0_kw * inverse(1 - U * chi0_kw)
    return chi_rpa

## Plot the MDC

In [ ]:
g0_kw = get_g0(beta)
chi0_kw = get_chi0(g0_kw)
chi_sp_kw = chi_rpa(chi0_kw, 2.0)

oplot(chi_sp_kw[Idx(2,2,0),:], '-o', x_window=[-10,10])

In [ ]:
g = four(chi_sp_kw)
oplot(g[Idx(2,2,0),:])